In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import openpyexcel
from openpyexcel import workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyexcel.utils import get_column_letter
from itertools import islice
from os import listdir
import re

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.0f}'.format

In [5]:
company = pd.read_csv('CSV/company_ws.csv',index_col=0)

In [8]:
income_sheets = pd.read_pickle('pickle/income_sheets.pkl')

In [10]:
income_sheets['CAT'][2]

,2018,2017,2016
Index,,,
Sales and revenues:,NaN,NaN,NaN
Sales and revenues,54722,45462,38537
Cost of goods sold,36997,31260,28044
Operating costs:,NaN,NaN,NaN
"Selling, general and administrative expenses",5478,4999,4383
Research and development expenses,1850,1842,1853
Interest expense of Financial Products,722,646,596
Goodwill impairment charge,0,0,595
Other operating (income) expenses,1382,2255,1904


In [62]:
def add_to_income_sheet1(table):
   
   d = pd.DataFrame()
   
   #k1 = table[table.index.str.contains(r"Sales and (Revenue|Revenues)$",case=False, regex=True,na=False)]

  # if len(k1) > 0:
   #  d["Total Revenue"] = k1.iloc[0,]
   
   k1 = table[table.index.str.contains(r".*Sales.*",case=False, regex=True,na=False)]
    
   if len(k1) > 0:
     d["Sales"] = k1.iloc[0,]
        
   k1 = table[table.index.str.contains(r".*Total.*revenue.*",case=False, regex=True,na=False)]
    
   if len(k1) > 0:
     d["Total Revenue"] = k1.iloc[0,]
        
   k1 = table[table.index.str.contains(r"Cost of goods sold",case=False, regex=True,na=False)]
    
   if len(k1) > 0:
     d["Cost of Sales"] = k1.iloc[0,]
   else:
     k1 = table[table.index.str.contains(r"Cost of Sales",case=False, regex=True,na=False)]
     
     if len(k1) > 0:
       d["Cost of Sales"] = k1.iloc[0,]
        
   k1 = table[table.index.str.contains(r"Selling, general and administrative",case=False, regex=True,na=False)]
    
   if len(k1) > 0:
     d["Selling, general and administrative"] = k1.iloc[0,]

   k1 = table[table.index.str.contains(r"Operating costs$",case=False, regex=True,na=False)]

   if len(k1) > 0:
      d["Operating Costs"] = k1.iloc[0,]
        
   k1 = table[table.index.str.contains(r"Total costs and expenses",case=False, regex=True,na=False)]

   if len(k1) > 0:
      d["Total costs and expenses"] = k1.iloc[0,]
       
   k1 = table[table.index.str.contains(r"Gross Profit",case=False, regex=True,na=False)]

   if len(k1) > 0:
      d["Gross Profit"] = k1.iloc[0,]
   else:
      k1 = table[table.index.str.contains(r"Operating Income",case=False, regex=True,na=False)]

   if len(k1) > 0:
     d["Operating Profit"] = k1.iloc[0,]
   else:
     k1 = table[table.index.str.contains(r"Operating Income",case=False, regex=True,na=False)]
    
     if len(k1) > 0:
       d["Operating Profit"] = k1.iloc[0,]
      
   return d

In [85]:
rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"
dirname = rootdir + 'DLTR'
f = 'DLTR1.xlsx'
path = dirname + '/' + f

xl_file = pd.ExcelFile(path)
table = xl_file.parse(company.loc[f,'Income'])

In [93]:
def clean_table (table):
    
    df = table.copy()
    
    if len(df.columns) > 8:
      # Take the first and the last 3 columns
        
      a = df.iloc[:,:1].copy()
      b = df.iloc[:,-3:].copy()
      df = pd.concat([a,b], axis=1).copy()
    else:
      df = table.copy()
        
    if df[df.columns[1]].dtype == object :
       df[df[df.columns[1]].str.contains((r"[\d]"), case=False, regex=True,na=False)] = np.nan
    
    df.iloc[0].fillna('',inplace=True)
    
    if 'unnamed' in df.columns[3].lower():
        # Column Names from first row
        
        col = list(df.iloc[0])
    else:
        col = list(df.columns[1:])
    
    for i in range(0,len(col)):
      col[i] = col[i][-4:]
    
    df.columns = col  
    df.drop(index=df.index[0], axis=0, inplace=True)
    df.set_index(df.columns[0], inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    return df

In [87]:
table

,CONSOLIDATED INCOME STATEMENTS - USD ($) $ in Millions,12 Months Ended,Unnamed: 2,Unnamed: 3
0,NaN,"Jan. 28, 2023","Jan. 29, 2022","Jan. 30, 2021"
1,Income Statement [Abstract],,,
2,Net sales,28318,26310,25508
3,Other revenue,14,11,1
4,Total revenue,28332,26321,25509
5,Cost of sales,19396,18584,17721
6,"Selling, general and administrative expenses",6699,5926,5900
7,Operating income,2236,1811,1888
8,"Interest expense, net",125,179,147
9,"Other expense, net",0,0,1


In [86]:
k = clean_table(table)
k

""
""
"(Income Statement [Abstract], , , )"
"(Net sales, 28318.2, 26309.8, 25508.4)"
"(Other revenue, 13.5, 11.4, 0.9)"
"(Total revenue, 28331.7, 26321.2, 25509.3)"
"(Cost of sales, 19396.3, 18583.9, 17721)"
"(Selling, general and administrative expenses, 6699.1, 5925.9, 5900.4)"
"(Operating income, 2236.3, 1811.4, 1887.9)"
"(Interest expense, net, 125.3, 178.9, 147.3)"
"(Other expense, net, 0.4, 0.3, 0.8)"


In [64]:
rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

inc_list = []

for d in listdir(rootdir):
  income_sheet = pd.DataFrame()
  dirname = rootdir + d
  for f in listdir(dirname):
    if f.endswith('xlsx'):
      print(f)
      path = dirname + '/' + f
    
      xl_file = pd.ExcelFile(path)
      table = xl_file.parse(company.loc[f,'Income'])
      ct = clean_table(table)
      df = add_to_income_sheet1(ct) 
      income_sheet = income_sheet.append(df)
  income_sheet = income_sheet.T
  income_sheet['company'] = d
  inc_list.append(income_sheet)

CAT1.xlsx
CAT2.xlsx
CAT3.xlsx
CAT4.xlsx
CAT5.xlsx
DG1.xlsx
DG2.xlsx
DG3.xlsx
DG4.xlsx
DG5.xlsx
DLTR1.xlsx
DLTR2.xlsx
DLTR3.xlsx
DLTR4.xlsx
DLTR5.xlsx
DUK1.xlsx
DUK2.xlsx
DUK3.xlsx
DUK4.xlsx
DUK5.xlsx
ED1.xlsx
ED2.xlsx
ED3.xlsx
ED4.xlsx
ED5.xlsx
EIX1.xlsx
EIX2.xlsx
EIX3.xlsx
EIX4.xlsx
EIX5.xlsx
TAP1.xlsx
TAP2.xlsx
TAP4.xlsx
TJX1.xlsx
TJX2.xlsx
TJX3.xlsx
TJX4.xlsx
TJX5.xlsx
TSLA1.xlsx
TSLA2.xlsx
TSLA3.xlsx
TSLA4.xlsx
TSLA5.xlsx
VZ1.xlsx
VZ2.xlsx
VZ3.xlsx
VZ4.xlsx
VZ5.xlsx
WAB1.xlsx
WAB2.xlsx
WAB3.xlsx
WAB4.xlsx
WAB5.xlsx
WEC1.xlsx
WEC2.xlsx
WEC3.xlsx
WEC4.xlsx
WEC5.xlsx
WMT1.xlsx
WMT2.xlsx
WMT3.xlsx
WMT4.xlsx
WMT5.xlsx
WYNN1.xlsx
WYNN2.xlsx
WYNN3.xlsx
WYNN4.xlsx
WYNN5.xlsx


In [75]:
inc_list[8]

,2019,2018,2017,2018,2017,2016,2015,2014,2013,company
Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TSLA
Total Revenue,20821,18515,9642,18514983,9641300,6350766,4046025,3198356,2013496,TSLA
"Selling, general and administrative",2646,2835,2477,2834491,2476500,1432189,922232,603660,285569,TSLA
Gross Profit,4069,4042,2223,4042021,2222487,1599257,923503,881671,456262,TSLA
Operating Profit,4069,4042,2223,4042021,2222487,1599257,923503,881671,456262,TSLA


In [92]:
table
#table.dropna(axis=1, how='all')

,CONSOLIDATED INCOME STATEMENTS - USD ($) $ in Millions,12 Months Ended,Unnamed: 2,Unnamed: 3
0,NaN,"Jan. 28, 2023","Jan. 29, 2022","Jan. 30, 2021"
1,Income Statement [Abstract],,,
2,Net sales,28318,26310,25508
3,Other revenue,14,11,1
4,Total revenue,28332,26321,25509
5,Cost of sales,19396,18584,17721
6,"Selling, general and administrative expenses",6699,5926,5900
7,Operating income,2236,1811,1888
8,"Interest expense, net",125,179,147
9,"Other expense, net",0,0,1


In [90]:
clean_table(table)

""
""
"(Income Statement [Abstract], , , )"
"(Net sales, 28318.2, 26309.8, 25508.4)"
"(Other revenue, 13.5, 11.4, 0.9)"
"(Total revenue, 28331.7, 26321.2, 25509.3)"
"(Cost of sales, 19396.3, 18583.9, 17721)"
"(Selling, general and administrative expenses, 6699.1, 5925.9, 5900.4)"
"(Operating income, 2236.3, 1811.4, 1887.9)"
"(Interest expense, net, 125.3, 178.9, 147.3)"
"(Other expense, net, 0.4, 0.3, 0.8)"


In [38]:
last_column = len(df.columns) - 1
df[df.columns[last_column]]
df.loc[df[df.columns[last_column]].str.contains((r".*[\d]"), regex=True, na=False)]

,Consolidated Statements of Earnings (USD $),3 Months Ended,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,12 Months Ended,Unnamed: 18,Unnamed: 19


In [53]:
df.set_index(df.columns[0], inplace=True)

In [55]:
df.columns = list(df.iloc[0])

In [70]:
clean_table(df)

TypeError: 'datetime.datetime' object is not subscriptable

In [28]:
len(col)

4

In [42]:
df.iloc[:,-3:]


,12 Months Ended,Unnamed: 18,Unnamed: 19
0,"Jan. 31, 2015","Feb. 01, 2014","Feb. 02, 2013"
1,NaN,NaN,NaN
2,11041677,10230353,9721065
3,NaN,NaN,NaN
4,7937956,7360924,7011428
5,1615371,1526366,1437886
6,2984,-247,6907
7,9556311,8887043,8456221
8,1485366,1343310,1264844
9,560642,506006,478081


In [46]:
a = df.iloc[:,:1]
b = df.iloc[:,-3:]
pd.concat([a,b], axis=1)

,Consolidated Statements of Earnings (USD $),12 Months Ended,Unnamed: 18,Unnamed: 19
0,"In Thousands, except Per Share data, unless ot...","Jan. 31, 2015","Feb. 01, 2014","Feb. 02, 2013"
1,Income Statement [Abstract],NaN,NaN,NaN
2,Sales,11041677,10230353,9721065
3,Costs and Expenses,NaN,NaN,NaN
4,Cost of goods sold,7937956,7360924,7011428
5,"Selling, general and administrative",1615371,1526366,1437886
6,"Interest expense (income), net",2984,-247,6907
7,Total costs and expenses,9556311,8887043,8456221
8,Earnings before taxes,1485366,1343310,1264844
9,Provision for taxes on earnings,560642,506006,478081


In [33]:
df

,,2019,2018,2017
0,,"Feb. 01, 2019","Feb. 02, 2018","Feb. 03, 2017"
1,CONSOLIDATED STATEMENTS OF INCOME,NaN,NaN,NaN
2,Net sales,25625043,23470967,21986598
3,Cost of goods sold,17821173,16249608,15203960
4,Gross profit,7803870,7221359,6782638
5,"Selling, general and administrative expenses",5687564,5213541,4719189
6,Operating profit,2116306,2007818,2063449
7,Interest expense,99871,97036,97821
8,Other (income) expense,1019,3502,NaN
9,Income before income taxes,2015416,1907280,1965628


In [34]:
df.drop(index=df.index[0], axis=0, inplace=True)

In [94]:
inc_list[5]

,2022,2021,2020,2020,2019,2018,2018,2017,2016,2016,2015,2014,2014,,2013,,2012,company
Total Revenue,17220,14905,13578,13578,12347,12657,12657,12320,11869,11869,11524,13413,13413,NaN,12581,NaN,11862,EIX
Operating Profit,-5,-2,-133,-133,-5,-7,-7,-9,0,2092,2008,2472,NaN,NaN,NaN,NaN,NaN,EIX


In [36]:
df.set_index(df.columns[0], inplace=True)

In [23]:
df.size

380

In [38]:
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

In [24]:
df.shape[1]

20